## Segmentação dos dados

As características normalmente são extraídas sobre pequenos segmentos de tamanho fixo dos dados, não no dado como um todo. Estes pequenos segmentos são chamados de janelas. A técnica de separar os dados em janelas, recebe o nome de janela deslizante (*sliding window*) e é uma forma de segmentação de dados. Uma boa prática, é definir um passo para essa janela de forma que haja uma sopreposição de dados, para que informações da lacuna de uma janela e outra não sejam perdidas. Na imagem a seguir podemos observar um sinal EOG. Abaixo dele a representação de janelas: *W1*, *W2*, *W3*... Repare que entre as janelas, há uma sobreposição de tamanho *T*. 

Na base de dados que iremos implementar, cada ensaio tem duração de 5 segundos. Se utilizadas janelas com tamanho de 250 ms, resultará em 20 janelas. Ao aplicar uma sobreposição de ~128ms, ficamos com 41 janelas de ~122ms.

Algumas características de EMG trabalham com o dado no domínio da frequência. Quando tais características são aplicadas, é necessário tranformar o dado para o domínio da frequência, utilizando o método de transformação de domínio `STFT`, do inglês *Short-time Fourier transform* (Transformada de Fourier de curto termo). O código divide o dado em segmentos, tanto no domínio do tempo quanto no domínio da frequência.

In [8]:
# Criando dados fictícios

import numpy as np

shape = (60, 4, 1000)
data = np.load('./datasets/Jesse.npy')
data.shape

(60, 4, 1000)

In [9]:
from scipy.signal import stft

step = 29
segment = 64
print('', data.shape)

n_win = int((data.shape[-1] - segment) / step) + 1
ids = np.arange(n_win) * step

# Janelas do dado no dominio do tempo
chunks_time = np.array([data[:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3)

# Janelas do dado no domínio da frequência
_, _, chunks_freq = stft(data, fs=200, nperseg=64, noverlap=32)
chunks_freq = np.swapaxes(chunks_freq, 2, 3)

print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Dominio do tempo: {chunks_time.shape} - (classes+ensaios, canais, janelas, linhas)')
print(f'Dominio da frequência:  {chunks_freq.shape} - (classes+ensaios, canais, janelas, linhas)')

 (60, 4, 1000)
Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (60, 4, 33, 64) - (classes+ensaios, canais, janelas, linhas)
Dominio da frequência:  (60, 4, 33, 33) - (classes+ensaios, canais, janelas, linhas)


*Desafio*: Implementar um algoritmo para encontrar a mesma quantidade de janelas em ambos os domínios (tempo e frequência).

In [10]:
import numpy as np
from scipy.signal import stft

def create_windows(data, segment, step, domain='time'):
    if domain == 'time':
        n_win = int((data.shape[-1] - segment) / step) + 1
        ids = np.arange(n_win) * step
        windows = np.array([data[:, :, k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3)
    elif domain == 'frequency':
        _, _, zxx = stft(data, fs=200, nperseg=segment, noverlap=step - 1)  # Adjusted parameters
        windows = np.swapaxes(zxx, 2, 3)
    else:
        return

    return windows


step = 1
segment = 64
chunks_time = None
chunks_freq = None

while True:
    chunks_time = create_windows(data, segment, step, domain='time')
    chunks_freq = create_windows(data, segment, step, domain='frequency')

    n_win_time = chunks_time.shape[2]
    n_win_freq = chunks_freq.shape[2]

    if n_win_time == n_win_freq:
        break
    else:
        step += 1

print("Step:", step)
print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Domínio do tempo: {chunks_time.shape} - (classes+ensaios, canais, janelas, linhas)')
print(f'Domínio da frequência:  {chunks_freq.shape} - (classes+ensaios, canais, janelas, linhas)')


Step: 31
Formato (shape) dos dados depois da divisão de janelas
Domínio do tempo: (60, 4, 31, 64) - (classes+ensaios, canais, janelas, linhas)
Domínio da frequência:  (60, 4, 31, 33) - (classes+ensaios, canais, janelas, linhas)
